In [1]:
#pip install av
#!pip install body-matrix==0.3.9

   ---------------------------------------- 25.8/25.8 MB 3.9 MB/s eta 0:00:00


  Attempting uninstall: body-matrix
    Found existing installation: body-matrix 0.4.4
    Uninstalling body-matrix-0.4.4:
      Successfully uninstalled body-matrix-0.4.4


In [2]:
#!pip install --upgrade body-matrix

  Using cached body_matrix-0.4.4-py3-none-any.whl.metadata (2.7 kB)
Using cached body_matrix-0.4.4-py3-none-any.whl (14 kB)
  Attempting uninstall: body-matrix
    Found existing installation: body-matrix 0.3.9
    Uninstalling body-matrix-0.3.9:
      Successfully uninstalled body-matrix-0.3.9


In [2]:
!pip install opencv-python
!pip install requests
!pip install body-matrix

In [16]:
import cv2
import requests
import numpy as np
from body_matrix import load, infer, process
from PIL import Image

In [10]:
IP_WEBCAM_URL = "http://192.168.29.199:8080/video"

In [11]:
keypoints_model, keypoints_transform = load.keypoints_model("cpu")
segment_model, segment_transform = load.segment_model("cpu")

In [46]:
def detect_body_parts_with_mask(frame):
    # Convert OpenCV image (NumPy array) to PIL image
    frame_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    
    try:
        # Detect main target with keypoints
        selected_box, keypoints = infer.detect_main_target(
            frame_pil, "cpu", 0.6, keypoints_model, keypoints_transform
        )
        
        if selected_box is None or keypoints is None:
            print("No person detected.")
            return None, False, None, None

        # Create segmentation mask
        mask, mask_image, bool_mask = infer.segment_selected_target(
            frame_pil, "cpu", selected_box, 0.99, segment_model, segment_transform
        )

        # Debugging output
        print(f"Mask: {type(mask)}, Mask Image: {type(mask_image)}, Bool Mask: {type(bool_mask)}")

        # Convert mask_image from PIL to NumPy array
        if mask_image is not None:
            mask_image = np.array(mask_image)

            if not isinstance(mask_image, np.ndarray):
                print("Mask image conversion failed.")
                return None, False, None, None
        else:
            print("Mask image is None.")
            return None, False, None, None

        # Ensure the mask has the same dimensions as the original frame
        if mask_image.shape[:2] != frame.shape[:2]:
            mask_image = cv2.resize(mask_image, (frame.shape[1], frame.shape[0]), interpolation=cv2.INTER_NEAREST)

        # Ensure the mask has three channels (RGB)
        if len(mask_image.shape) == 2:  # If mask is grayscale
            mask_image = cv2.cvtColor(mask_image, cv2.COLOR_GRAY2BGR)

        # Apply the pixelated mask as an overlay with semi-transparency
        colored_frame = cv2.addWeighted(frame, 0.7, mask_image, 0.3, 0)

        # Draw keypoints on the colored_frame
        if keypoints is not None:
            for keypoint in keypoints:
                x, y = int(keypoint[0]), int(keypoint[1])
                if 0 <= x < colored_frame.shape[1] and 0 <= y < colored_frame.shape[0]:  # Check bounds
                    cv2.circle(colored_frame, (x, y), 5, (0, 255, 0), -1)  # Draw keypoints as green dots

        # Return detected parts and overall detection success along with mask and keypoints
        return colored_frame, True, mask_image, keypoints

    except Exception as e:
        print(f"Error during body part detection: {e}")
        return None, False, None, None

In [42]:
def estimate_body_dimensions(keypoints):
    # Convert keypoints from tensors to NumPy arrays
    HEAD = keypoints[0].detach().numpy()        # Example index for head
    NECK = keypoints[1].detach().numpy()        # Example index for neck
    LEFT_SHOULDER = keypoints[2].detach().numpy() # Example index for left shoulder
    RIGHT_SHOULDER = keypoints[3].detach().numpy() # Example index for right shoulder
    LEFT_ELBOW = keypoints[4].detach().numpy()  # Example index for left elbow
    RIGHT_ELBOW = keypoints[5].detach().numpy()  # Example index for right elbow
    LEFT_HAND = keypoints[6].detach().numpy()   # Example index for left hand
    RIGHT_HAND = keypoints[7].detach().numpy()  # Example index for right hand
    HIPS = keypoints[8].detach().numpy()        # Example index for hips
    LEFT_KNEE = keypoints[9].detach().numpy()   # Example index for left knee
    RIGHT_KNEE = keypoints[10].detach().numpy() # Example index for right knee
    LEFT_FOOT = keypoints[11].detach().numpy()  # Example index for left foot
    RIGHT_FOOT = keypoints[12].detach().numpy() # Example index for right foot

    # Calculate dimensions
    height = np.linalg.norm(HEAD - LEFT_FOOT)  # Head to foot
    shoulder_width = np.linalg.norm(LEFT_SHOULDER - RIGHT_SHOULDER)  # Shoulders
    hand_length = np.linalg.norm(LEFT_HAND - LEFT_ELBOW)  # Left hand length
    waist_size = shoulder_width * 0.8  # Approximation; replace with more accurate measurement if available
    hip_width = np.linalg.norm(HIPS[0] - HIPS[1])  # Left to right hip

    # Store dimensions in a dictionary
    dimensions = {
        "Height": height,
        "Shoulder Width": shoulder_width,
        "Hand Length": hand_length,
        "Waist Size": waist_size,
        "Hip Width": hip_width,
    }

    return dimensions


In [48]:
cap = cv2.VideoCapture(IP_WEBCAM_URL)

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to retrieve frame.")
        break
    
    # Resize frame for faster processing
    frame_resized = cv2.resize(frame, (640, 480))

    # Detect body parts with pixelated mask overlay
    masked_frame, all_parts_detected, pixelated_mask, keypoints = detect_body_parts_with_mask(frame_resized)

    if masked_frame is not None:
        cv2.imshow("Input Frame", frame)

        # Show frame with pixelated mask overlay
        cv2.imshow("Pixelated Body Segmentation Mask", masked_frame)

        # Stop capture if all body parts are detected
        if all_parts_detected and keypoints is not None:
            print("All body segments identified.")
            dimensions = estimate_body_dimensions(keypoints)  # Ensure this function is defined
            print("Estimated Body Dimensions:", dimensions)
            break

    # Press 'q' to exit manually
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture and close display window
cap.release()
cv2.destroyAllWindows()

Found  1  person in the frame
Mask: <class 'torch.Tensor'>, Mask Image: <class 'PIL.Image.Image'>, Bool Mask: <class 'torch.Tensor'>
All body segments identified.
Estimated Body Dimensions: {'Height': 149.33458, 'Shoulder Width': 54.268955, 'Hand Length': 58.32308, 'Waist Size': 43.415164184570315, 'Hip Width': 66.63896}
